In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importing DataFrame
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None) 
df_ = pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
df = df_.drop(['id'], axis=1)
df

# Perform a sweetviz analysis as that will give you most EDA information in one line
<!-- # Analyzing data -->
report=sv.analyze(df)
<!-- # Generating report -->
report.show_html('report.html')

> There is one "Other" gender since it does give us much info get rid of it
> > Do the dep and indep variable split

In [ ]:
df.drop(df[df['gender'] == 'Other'].index, inplace = True)
y = df['stroke']
df.drop(['stroke'], axis=1, inplace=True)

In [ ]:
df.isnull().sum()

# There are null values in BMI so do the mean Imputing

In [ ]:
# Impute BMI
df['bmi'].fillna((df['bmi'].mean()), inplace=True)

# See a Pairplot and understand the Data distribution


In [ ]:
import seaborn as sns
sns.pairplot(data=df_, hue="Residence_type")

# Doing One hot Encoding always with a Sklearn package is not a good thing because we miss some level of fine tuning.
Here I decided to drop "never_worked" from "work_type" and "Unknown" from "Smoking" rather than dropping the first column always.

Do the minMaxScaling on Age, Glucose_level, BMI

Change Gender, Maried, Residence_type from Object to label encoded Int

Then drop the unnecessary columns

In [ ]:
# Reframing and removing Gender
df.loc[(df.gender == 'Male'),'gender'] = 1
df.loc[(df.gender == 'Female'),'gender'] = 0


# Reframing and removing Married
df.loc[(df.ever_married == 'Yes'),'ever_married'] = 1
df.loc[(df.ever_married == 'No'),'ever_married'] = 0

# Reframing and removing residenceType
df.loc[(df.Residence_type == 'Urban'),'Residence_type'] = 1
df.loc[(df.Residence_type == 'Rural'),'Residence_type'] = 0

from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

# Scale Age, GlucoseLevel, Bmi
df[['age', 'avg_glucose_level', 'bmi']] = min_max_scaler.fit_transform(df[['age', 'avg_glucose_level', 'bmi']])

# Create columns for work type
df[['children', 'govt_job', 'private', 'self_employed']] = 0


# Filling based on condition
for index, row in df.iterrows():
    if row['work_type'] == 'Private':
        row['private'] = 1
    elif row['work_type'] == 'Self-employed':
        row['self_employed'] = 1
    elif row['work_type'] == 'children':
        row['children'] = 1
    elif row['work_type'] == 'Govt_job':
        row['govt_job'] = 1
        
df.drop(['work_type'], axis=1, inplace=True)        
        
# Create columns for smoking status
df[['never_smoked', 'formerly_smoked', 'smoked']] = 0


# Filling based on condition
for index, row in df.iterrows():
    if row['smoking_status'] == 'never smoked':
        row['never_smoked'] = 1
    elif row['smoking_status'] == 'formerly smoked':
        row['formerly_smoked'] = 1
    elif row['smoking_status'] == 'smokes':
        row['smoked'] = 1
        
df.drop(['smoking_status'], axis=1, inplace=True)

In [ ]:
df.dtypes

**Although you change the data column data type will not change so we have to apply those changes also.**

In [ ]:
df[["gender", "ever_married", "Residence_type"]] = df[["gender", "ever_married", "Residence_type"]].apply(pd.to_numeric)

# Since we are working with highly imbalanced data we have to do startified split to understand the dynamics

In [ ]:
# Test train Split with Stratify

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=0, stratify=y)

# Let's implement and see various algorithms to pick the best one

**Implementing KNN**


In [ ]:
# Testing with KNN Algorithm
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()
neigh.fit(x_train, y_train)

knn_pred = neigh.predict(x_test)
from sklearn import metrics
print(metrics.plot_confusion_matrix(neigh, x_test, y_test))
print(metrics.classification_report(y_test, knn_pred, digits=3))

# KNN report

Accuracy is high.. But this is an imbalanced set so focus on other aspects like Precision, Recall, and F1 Scores...

For 0's it's ok But that's not the case for 1's

So, this is not recommended

# Implementing Gaussian Naive Bayes

In [ ]:
# Testing with GNB Algorithm
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB()
GNB.fit(x_train, y_train)

gnb_pred = GNB.predict(x_test)
from sklearn import metrics
print(metrics.plot_confusion_matrix(GNB, x_test, y_test))
print(metrics.classification_report(y_test, gnb_pred, digits=3))

**More or Less GNB is also falling in the same category. So, there is something critically wrong with the data**

**It's not our steps of preprocessing but the distribution of data.**

So, we need to correct this imbalance and we can do that using SMOTE from imblearn package

In [ ]:
# Since its doing worst lets try smote and retrain
!pip install imblearn
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_resampled, y_resampled = oversample.fit_resample(df, y)

# Now we have the Smoted data Let's see its work on six Algorithms

In [ ]:
# Test train Split with Stratify

from sklearn.model_selection import train_test_split

x_train_resampled, x_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled,
                                                    test_size=0.33,
                                                    random_state=0,
                                                    stratify=y_resampled)

# Applying KNN on Smoted Data

In [ ]:
# Testing with KNN Algorithm after smote
from sklearn.neighbors import KNeighborsClassifier
neigh2 = KNeighborsClassifier()
neigh2.fit(x_train_resampled, y_train_resampled)

knn_pred_resampled = neigh2.predict(x_test_resampled)
from sklearn import metrics
print(metrics.plot_confusion_matrix(neigh2, x_test_resampled, y_test_resampled))
print(metrics.classification_report(y_test_resampled, knn_pred_resampled, digits=3))

**We can immediately notice that there is little reduction in Accuracy**
**Along with that we also need to make note that there is a significant raise in Precision, Recall, and F1-Scores. So, this is much more sophisticated than the previous models**

# Applying GNB on Smoted Data

In [ ]:
# Testing with GNB Algorithm after smote
from sklearn.naive_bayes import GaussianNB
GNB2 = GaussianNB()
GNB2.fit(x_train_resampled, y_train_resampled)

gnb_pred_resampled = GNB2.predict(x_test_resampled)
from sklearn import metrics
print(metrics.plot_confusion_matrix(GNB2, x_test_resampled, y_test_resampled))
print(metrics.classification_report(y_test_resampled, gnb_pred_resampled, digits=3))

# Applying Logistic Regression on Smoted Data

In [ ]:
# Testing with Logistic Regression Algorithm after smote
from sklearn.linear_model import LogisticRegression
LR2 = LogisticRegression()
LR2.fit(x_train_resampled, y_train_resampled)

lr_pred_resampled = LR2.predict(x_test_resampled)
from sklearn import metrics
print(metrics.plot_confusion_matrix(LR2, x_test_resampled, y_test_resampled))
print(metrics.classification_report(y_test_resampled, lr_pred_resampled, digits=3))

# Applying Decision Tree on Smoted Data

In [ ]:
# Testing with Decision Tree Algorithm after smote
from sklearn.tree import DecisionTreeClassifier
DT2 = DecisionTreeClassifier()
DT2.fit(x_train_resampled, y_train_resampled)

dt_pred_resampled = DT2.predict(x_test_resampled)
from sklearn import metrics
print(metrics.plot_confusion_matrix(DT2, x_test_resampled, y_test_resampled))
print(metrics.classification_report(y_test_resampled, dt_pred_resampled, digits=3))

# Applying random Forest on Smoted Data

In [ ]:
# Testing with Random Forest Algorithm after smote
from sklearn.ensemble import RandomForestClassifier
RF2 = RandomForestClassifier()
RF2.fit(x_train_resampled, y_train_resampled)

rf_pred_resampled = RF2.predict(x_test_resampled)
from sklearn import metrics
print(metrics.plot_confusion_matrix(RF2, x_test_resampled, y_test_resampled))
print(metrics.classification_report(y_test_resampled, rf_pred_resampled, digits=3))

# Applying Support Vector Machine on Smoted Data

In [ ]:
# Testing with SVC Algorithm after smote
from sklearn.svm import SVC
SVC2 = SVC()
SVC2.fit(x_train_resampled, y_train_resampled)

svc_pred_resampled = SVC2.predict(x_test_resampled)
from sklearn import metrics
print(metrics.plot_confusion_matrix(SVC2, x_test_resampled, y_test_resampled))
print(metrics.classification_report(y_test_resampled, svc_pred_resampled, digits=3))

# By seeing all the Confusion matrices and Classification Reports we cacn strongly say that Decision Tree and random Forest are performing great.
**On top of that Random forest is False Negatives So it's better to opt Random Forest. Because it is better to alert a normal person than not alerting a patient**

**Lets place all the matrices in one plot and consider this for review**

In [ ]:
from matplotlib import pyplot as plt
from sklearn import metrics
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15,10))

classifiers = [neigh2, GNB2, LR2, DT2, RF2, SVC2]

for cls, ax in zip(classifiers, axes.flatten()):
    metrics.plot_confusion_matrix(cls, x_test_resampled, y_test_resampled, 
                          ax=ax)
    ax.title.set_text(type(cls).__name__)
plt.tight_layout()
plt.show()

# the End

In [ ]:
|